## Twitter sentiment analysis of airlines:

In [1]:
import numpy as np
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/twitter-airline-sentiment/Tweets.csv
/kaggle/input/twitter-airline-sentiment/database.sqlite


In [2]:
tweet_data = pd.read_csv('/kaggle/input/twitter-airline-sentiment/Tweets.csv')
print("data shape:",tweet_data.shape)
print("what are columns:",tweet_data.columns)
tweet_data.head()

data shape: (14640, 15)
what are columns: Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


## Data Cleaning

In [3]:
tweet_data = tweet_data.drop(['tweet_id','retweet_count', 'tweet_coord', 'tweet_created',
                               'tweet_location','name','user_timezone'],axis = 1)

In [4]:
tweet_data.head()

,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,negativereason_gold,text
0,neutral,1.0000,NaN,NaN,Virgin America,NaN,NaN,@VirginAmerica What @dhepburn said.
1,positive,0.3486,NaN,0.0000,Virgin America,NaN,NaN,@VirginAmerica plus you've added commercials t...
2,neutral,0.6837,NaN,NaN,Virgin America,NaN,NaN,@VirginAmerica I didn't today... Must mean I n...
3,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,NaN,@VirginAmerica it's really aggressive to blast...
4,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,NaN,@VirginAmerica and it's a really big bad thing...


In [5]:
tweet_data['negativereason_gold'].unique()

array([nan, 'Late Flight\nFlight Attendant Complaints',
       'Late Flight\nLost Luggage', 'Late Flight',
       'Customer Service Issue', "Can't Tell",
       'Cancelled Flight\nCustomer Service Issue', 'Cancelled Flight',
       'Bad Flight', 'Lost Luggage\nDamaged Luggage',
       'Late Flight\nCancelled Flight', 'Flight Attendant Complaints',
       'Customer Service Issue\nLost Luggage',
       "Customer Service Issue\nCan't Tell"], dtype=object)

In [6]:
tweet_data['negativereason'] = tweet_data['negativereason'].fillna('')
tweet_data['negativereason_confidence'] = tweet_data['negativereason_confidence'].fillna(0)
tweet_data['airline_sentiment_gold'] = tweet_data['airline_sentiment_gold'].fillna('')
tweet_data['negativereason_gold'] = tweet_data['negativereason_gold'].fillna('')

In [7]:
tweet_data.head()

,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,negativereason_gold,text
0,neutral,1.0000,,0.0000,Virgin America,,,@VirginAmerica What @dhepburn said.
1,positive,0.3486,,0.0000,Virgin America,,,@VirginAmerica plus you've added commercials t...
2,neutral,0.6837,,0.0000,Virgin America,,,@VirginAmerica I didn't today... Must mean I n...
3,negative,1.0000,Bad Flight,0.7033,Virgin America,,,@VirginAmerica it's really aggressive to blast...
4,negative,1.0000,Can't Tell,1.0000,Virgin America,,,@VirginAmerica and it's a really big bad thing...


In [8]:
print("different topics of negative reasons are:",tweet_data['negativereason'].unique())

different topics of negative reasons are: ['' 'Bad Flight' "Can't Tell" 'Late Flight' 'Customer Service Issue'
 'Flight Booking Problems' 'Lost Luggage' 'Flight Attendant Complaints'
 'Cancelled Flight' 'Damaged Luggage' 'longlines']


In [9]:
!pip install contractions

     |████████████████████████████████| 321 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 260 kB 5.2 MB/s eta 0:00:01
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=102921 sha256=cd65a463785362f61d5072289f07ac4d2568d8a6a1b1e445c439c67c6879fbe7
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [10]:
from nltk.corpus import stopwords
import string
import re
import contractions
def text_cleaning(text):
    #not removing the stopwords so that the sentences stay normal.
    #forbidden_words = set(stopwords.words('english'))
    if text:
        text = contractions.fix(text)
        text = ' '.join(text.split('.'))
        text = re.sub(r'\s+', ' ', re.sub('[^A-Za-z0-9]', ' ', text.strip().lower())).strip()
        text = re.sub(r'\W+', ' ', text.strip().lower()).strip()
        text = [word for word in text.split()]
        return text
    return []

In [11]:
tweet_data['text'] = tweet_data['text'].apply(lambda x: ' '.join(text_cleaning(x)))

In [12]:
tweet_data.head(20)

,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,negativereason_gold,text
0,neutral,1.0000,,0.0000,Virgin America,,,virginamerica what dhepburn said
1,positive,0.3486,,0.0000,Virgin America,,,virginamerica plus you have added commercials ...
2,neutral,0.6837,,0.0000,Virgin America,,,virginamerica i did not today must mean i need...
3,negative,1.0000,Bad Flight,0.7033,Virgin America,,,virginamerica it is really aggressive to blast...
4,negative,1.0000,Can't Tell,1.0000,Virgin America,,,virginamerica and it is a really big bad thing...
5,negative,1.0000,Can't Tell,0.6842,Virgin America,,,virginamerica seriously would pay 30 a flight ...
6,positive,0.6745,,0.0000,Virgin America,,,virginamerica yes nearly every time i fly vx t...
7,neutral,0.6340,,0.0000,Virgin America,,,virginamerica really missed a prime opportunit...
8,positive,0.6559,,0.0000,Virgin America,,,virginamerica well i did not but now i do d
9,positive,1.0000,,0.0000,Virgin America,,,virginamerica it was amazing and arrived an ho...


## Sentiment Analysis with NLTK

In [13]:
import nltk

In [14]:
nltk.download(["names","stopwords","state_union","twitter_samples",
              "movie_reviews","averaged_perceptron_tagger","vader_lexicon",
              "punkt"])

[nltk_data] Downloading package names to /usr/share/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to /usr/share/nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-

True

In [15]:
from nltk.sentiment import SentimentIntensityAnalyzer as SIA
sia = SIA()
print(sia.polarity_scores('wow! this nltk library really works'))

{'neg': 0.0, 'neu': 0.55, 'pos': 0.45, 'compound': 0.6239}


/opt/conda/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


Appending this sentiment scores to our original dataset. 

In [16]:
texts = tweet_data['text'].tolist()
negative_scores = []
neutral_scores = []
positive_scores = []
compound_scores = []
final_tag = []
for text in texts:
    score_dictionary = sia.polarity_scores(text)
    negative_scores.append(score_dictionary['neg'])
    positive_scores.append(score_dictionary['pos'])
    neutral_scores.append(score_dictionary['neu'])
    compound_scores.append(score_dictionary['compound'])
    if score_dictionary['compound']>0:
        final_tag.append('positive')
    elif score_dictionary['compound']<0:
        final_tag.append('negative')
    else:
        final_tag.append('neutral')
tweet_data['negative_score'] = negative_scores
tweet_data['positive_score'] = positive_scores
tweet_data['neutral_score'] = neutral_scores
tweet_data['compound_score'] = compound_scores
tweet_data['final_tag'] = final_tag

In [17]:
tweet_data.head(20)

,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,negativereason_gold,text,negative_score,positive_score,neutral_score,compound_score,final_tag
0,neutral,1.0000,,0.0000,Virgin America,,,virginamerica what dhepburn said,0.000,0.000,1.000,0.0000,neutral
1,positive,0.3486,,0.0000,Virgin America,,,virginamerica plus you have added commercials ...,0.000,0.000,1.000,0.0000,neutral
2,neutral,0.6837,,0.0000,Virgin America,,,virginamerica i did not today must mean i need...,0.000,0.000,1.000,0.0000,neutral
3,negative,1.0000,Bad Flight,0.7033,Virgin America,,,virginamerica it is really aggressive to blast...,0.216,0.123,0.661,-0.2716,negative
4,negative,1.0000,Can't Tell,1.0000,Virgin America,,,virginamerica and it is a really big bad thing...,0.296,0.000,0.704,-0.5829,negative
5,negative,1.0000,Can't Tell,0.6842,Virgin America,,,virginamerica seriously would pay 30 a flight ...,0.238,0.069,0.693,-0.5945,negative
6,positive,0.6745,,0.0000,Virgin America,,,virginamerica yes nearly every time i fly vx t...,0.000,0.172,0.828,0.4019,positive
7,neutral,0.6340,,0.0000,Virgin America,,,virginamerica really missed a prime opportunit...,0.142,0.175,0.683,0.1458,positive
8,positive,0.6559,,0.0000,Virgin America,,,virginamerica well i did not but now i do d,0.000,0.205,0.795,0.1406,positive
9,positive,1.0000,,0.0000,Virgin America,,,virginamerica it was amazing and arrived an ho...,0.000,0.340,0.660,0.7717,positive


In [19]:
from sklearn.metrics import classification_report as crep
print("sentiment analysis performance for nltk:")
print(crep(tweet_data['airline_sentiment'],tweet_data['final_tag']))

sentiment analysis performance for nltk:
              precision    recall  f1-score   support

    negative       0.90      0.45      0.60      9178
     neutral       0.43      0.32      0.37      3099
    positive       0.27      0.90      0.42      2363

    accuracy                           0.49     14640
   macro avg       0.53      0.56      0.46     14640
weighted avg       0.70      0.49      0.52     14640



## <a id = 'blob'>Sentiment analysis with Textblob</a>:


In [20]:
!pip install -U textblob
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /usr/share/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /usr/share/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [21]:
from textblob import TextBlob

In [22]:
tb1 = TextBlob('I just am trying textblob first time.')
tb1

TextBlob("I just am trying textblob first time.")

### tokenization and sentence segmentation:

In [23]:
tb1.words

WordList(['I', 'just', 'am', 'trying', 'textblob', 'first', 'time'])

In [24]:
tb1.sentences

[Sentence("I just am trying textblob first time.")]

In [25]:
tb1.noun_phrases

WordList([])

### pos tagging:

In [26]:
tb2=TextBlob("Tags will give the Part of speech for all the words.")
tb2.tags

[('Tags', 'NNP'),
 ('will', 'MD'),
 ('give', 'VB'),
 ('the', 'DT'),
 ('Part', 'NNP'),
 ('of', 'IN'),
 ('speech', 'NN'),
 ('for', 'IN'),
 ('all', 'PDT'),
 ('the', 'DT'),
 ('words', 'NNS')]

In [27]:
tb3=TextBlob(" We are learning cool Library . We are enjoying a lot .")
tb3.noun_phrases

WordList(['cool library'])

In [28]:
type(tb3.noun_phrases)

textblob.blob.WordList

### Polarity:
Polarity as discussed earlier helps us in finding the expression and emotion of the author in the text<br/>

In [29]:
doc2=TextBlob("We are having fun here")
doc2.polarity

0.3

In [30]:
texts = tweet_data['text'].tolist()
textblob_score = []
textblob_tag = []
for text in texts:
    doc_current = TextBlob(text)
    score = doc_current.polarity
    textblob_score.append(score)
    if score > 0:
        textblob_tag.append('positive')
    elif score<0:
        textblob_tag.append('negative')
    else:
        textblob_tag.append('neutral')
tweet_data['textblob_score'] = textblob_score
tweet_data['textblob_sentiment_tag'] = textblob_tag

In [31]:
tweet_data[['airline_sentiment','text','textblob_score','textblob_sentiment_tag']].head(20)

,airline_sentiment,text,textblob_score,textblob_sentiment_tag
0,neutral,virginamerica what dhepburn said,0.000000,neutral
1,positive,virginamerica plus you have added commercials ...,0.000000,neutral
2,neutral,virginamerica i did not today must mean i need...,-0.312500,negative
3,negative,virginamerica it is really aggressive to blast...,0.006250,positive
4,negative,virginamerica and it is a really big bad thing...,-0.350000,negative
5,negative,virginamerica seriously would pay 30 a flight ...,-0.208333,negative
6,positive,virginamerica yes nearly every time i fly vx t...,0.450000,positive
7,neutral,virginamerica really missed a prime opportunit...,0.200000,positive
8,positive,virginamerica well i did not but now i do d,0.000000,neutral
9,positive,virginamerica it was amazing and arrived an ho...,0.466667,positive


In [32]:
print("sentiment analysis with textblob:")
print(crep(tweet_data['airline_sentiment'],tweet_data['textblob_sentiment_tag']))

sentiment analysis with textblob:
              precision    recall  f1-score   support

    negative       0.88      0.35      0.50      9178
     neutral       0.32      0.57      0.41      3099
    positive       0.33      0.75      0.45      2363

    accuracy                           0.46     14640
   macro avg       0.51      0.56      0.46     14640
weighted avg       0.67      0.46      0.48     14640



## Huggingface Transformer based sentiment analysis

In [33]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')
classifier("I am so happy to use huggingface today!")[0]['label']

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

'POSITIVE'

In [34]:
classifier("it was a extremely bad movie!")[0]['label']

'NEGATIVE'

In [35]:
classifier("it is a statement")[0]['label']

'POSITIVE'

In [36]:
tweet_data['bert_based_sentiment'] = tweet_data['text'].apply(lambda x: classifier(x)[0]['label'].lower())
tweet_data_cut = tweet_data[tweet_data['airline_sentiment']!='neutral']
print(crep(tweet_data_cut['airline_sentiment'],tweet_data_cut['bert_based_sentiment']))

              precision    recall  f1-score   support

    negative       0.95      0.91      0.93      9178
    positive       0.69      0.81      0.75      2363

    accuracy                           0.89     11541
   macro avg       0.82      0.86      0.84     11541
weighted avg       0.90      0.89      0.89     11541



In [37]:
classifier_febu = pipeline('sentiment-analysis',model = 'facebook/bart-large')
classifier_febu("I hate it!")
classifier_febu("I love you!")

Downloading:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-large and are newly initialized: ['classification_head.dense.bias', 'classification_head.out_proj.weight', 'classification_head.out_proj.bias', 'classification_head.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'label': 'LABEL_0', 'score': 0.3882845342159271}]

In [38]:
def sentiment_analysis(text):
    sentiment = classifier_febu(text)[0]['label']
    if sentiment == 'LABEL_0': return 'positive'
    return 'negative'

In [39]:
tweet_data['facebook_bert_based_sentiment'] = tweet_data['text'].apply(sentiment_analysis)
tweet_data_cut = tweet_data[tweet_data['airline_sentiment']!='neutral']
print(crep(tweet_data_cut['airline_sentiment'],tweet_data_cut['facebook_bert_based_sentiment']))

              precision    recall  f1-score   support

    negative       0.79      0.98      0.88      9178
    positive       0.14      0.01      0.02      2363

    accuracy                           0.78     11541
   macro avg       0.47      0.50      0.45     11541
weighted avg       0.66      0.78      0.70     11541



In [40]:
classifier_third = pipeline('sentiment-analysis',model = 'cardiffnlp/twitter-roberta-base-sentiment')
print(classifier_third("I hate it!"))
print(classifier_third("I love you!"))
print(classifier_third("I don't know about the routine"))
def sentiment_analysis_third(text):
    sentiment = classifier_third(text)[0]['label']
    if sentiment == 'LABEL_0': return 'negative'
    elif sentiment == 'LABEL_1': return 'neutral'
    elif sentiment == 'LABEL_2': return 'positive'
    return 'neutral'
tweet_data['facebook_bert_based_sentiment'] = tweet_data['text'].apply(sentiment_analysis_third)
print(crep(tweet_data['airline_sentiment'],tweet_data['facebook_bert_based_sentiment']))

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

[{'label': 'LABEL_0', 'score': 0.9588500261306763}]
[{'label': 'LABEL_2', 'score': 0.9871658086776733}]
[{'label': 'LABEL_1', 'score': 0.5845447778701782}]
              precision    recall  f1-score   support

    negative       0.94      0.68      0.79      9178
     neutral       0.45      0.75      0.57      3099
    positive       0.69      0.83      0.76      2363

    accuracy                           0.72     14640
   macro avg       0.70      0.76      0.70     14640
weighted avg       0.80      0.72      0.74     14640



## Sentiment analysis with [Flair](https://github.com/flairNLP/flair) NLP framework for pretrained models:


In [42]:
!pip install flair

     |████████████████████████████████| 284 kB 1.3 MB/s eta 0:00:01     |███████████▌                    | 102 kB 1.3 MB/s eta 0:00:01     |███████████████████▋            | 174 kB 1.3 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 63 kB 1.2 MB/s eta 0:00:011
     |████████████████████████████████| 981 kB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 788 kB 6.6 MB/s eta 0:00:01
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9681 sha256=f33af68d134d20d2ce8ca1e594fa57255722ef3f416e86b32336e28c45b52730
  Stored in directory: /root/.cache/pip/wheels/ba/e0/7e/726e872a53f7358b4b96a9975b04e98113b005cd8609a63abc
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116678 sha256=59e701b25e61d8712a0f2f7b96cf45f3fba24a33f1fe8d322279a69f31a140e8
  Stored in directory: /root/.cache/pip/wheels/26/70/6a/1c79e59951

In [43]:
from flair.models import TextClassifier
from flair.data import Sentence
classifier = TextClassifier.load('en-sentiment')
sentence = Sentence('Flair is pretty neat!')
classifier.predict(sentence)
# print sentence with predicted labels
print('Sentence above is: ', sentence.labels)

2021-04-09 21:23:16,393 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmpcyaeae15


100%|██████████| 265512723/265512723 [00:16<00:00, 16318595.76B/s]

2021-04-09 21:23:33,198 copying /tmp/tmpcyaeae15 to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2021-04-09 21:23:33,705 removing temp file /tmp/tmpcyaeae15
2021-04-09 21:23:33,738 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Sentence above is:  [POSITIVE (0.9997)]


In [53]:
print(str(sentence.labels[0]).split("(")[0].lower()[:-1])

positive


In [52]:
def sentiment_analysis_flair(text):
    sentence = Sentence(text)
    classifier.predict(sentence)
    return str(sentence.labels[0]).split("(")[0].lower()[:-1]

In [55]:
tweet_data['flair_based_sentiment'] = tweet_data['text'].apply(sentiment_analysis_flair)
tweet_data_cut = tweet_data[tweet_data['airline_sentiment']!='neutral']
print(crep(tweet_data_cut['airline_sentiment'],tweet_data_cut['flair_based_sentiment']))

              precision    recall  f1-score   support

    negative       0.96      0.81      0.88      9178
    positive       0.55      0.88      0.68      2363

    accuracy                           0.83     11541
   macro avg       0.76      0.85      0.78     11541
weighted avg       0.88      0.83      0.84     11541

